In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score
import optuna
import lightgbm as lgb
import sklearn
import numpy as np

c:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [405]:
# read the data from /input/train.csv
data = pd.read_csv("train.csv")

In [406]:
X = data.drop(['Class', 'id'], axis=1)
y = data['Class']

In [407]:
X.head()

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512
1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251
2,112.640625,39.818393,0.379639,0.922306,2.730769,15.689690,8.193471,85.649785
3,120.679688,45.918448,-0.098490,0.011775,2.696488,20.954662,8.183874,70.332899
4,134.070312,57.720107,-0.107772,-0.573335,1.107860,11.255051,16.107748,308.753765


# Preprocessing with PolynomialFeatures!!!

In [390]:
# preprocess the data by adding interaction features
#from sklearn.preprocessing import PolynomialFeatures
#poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
#X = poly.fit_transform(X)

In [435]:
type(X)

pandas.core.frame.DataFrame

# Train, Val, Test Split

In [391]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.11, random_state=42)

In [438]:
X

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512
1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251
2,112.640625,39.818393,0.379639,0.922306,2.730769,15.689690,8.193471,85.649785
3,120.679688,45.918448,-0.098490,0.011775,2.696488,20.954662,8.183874,70.332899
4,134.070312,57.720107,-0.107772,-0.573335,1.107860,11.255051,16.107748,308.753765
...,...,...,...,...,...,...,...,...
117559,132.843750,56.748838,-0.060070,-0.554084,4.054348,27.844144,6.564423,44.442664
117560,112.578125,52.539271,0.179580,-0.306961,1.637960,15.331913,11.384718,142.535470
117561,119.757812,49.980013,0.064402,-0.270822,3.877926,19.788559,6.959740,56.367789
117562,105.789062,46.986595,0.441426,0.372466,2.097826,17.170612,9.442445,99.074539


In [437]:
X_train.shape

(94168, 36)

# Parameter search with Optuna

In [417]:
def objective(trial): 
    params = {
    'task': 'train', 
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':'binary_logloss', 
    'verbose':-10000000,
    'seed':42,
    'max_bin':128,
    'n_estimators':128,
    # 'n_estimators':trial.suggest_int("n_estimators", 800, 1200),
    'learning_rate':0.08,
    'feature_fraction':1.0,
    'bagging_fraction':1.0,
   # 'bagging_freq':trial.suggest_categorical("bagging_freq", [1]),
    'max_depth':trial.suggest_int("max_depth", 5, 12), # decrease this in the next round
    'num_leaves':trial.suggest_int("num_leaves",8, 32),
    'min_data_in_leaf':200,
    'min_gain_to_split':1.0,
}
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100)
    y_val_pred = model.predict_proba(X_val)
    logloss = log_loss(y_val, y_val_pred)
    return logloss

In [418]:
study = optuna.create_study(direction='minimize');
study.optimize(objective, n_trials=100);
optuna.visualization.plot_optimization_history(study)


[I 2023-03-12 23:41:31,747] A new study created in memory with name: no-name-a7544e16-f8a6-4ce5-8ef6-57bf7a0c54d0
c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split is set=1.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.246366
[2]	valid_0's binary_logloss: 0.21222
[3]	valid_0's binary_logloss: 0.18809
[4]	valid_0's binary_logloss: 0.169243
[5]	valid_0's binary_logloss: 0.153782
[6]	valid_0's binary_logloss: 0.140813
[7]	valid_0's binary_logloss: 0.129682
[8]	valid_0's binary_logloss: 0.119967
[9]	valid_0's binary_logloss: 0.111444
[10]	valid_0's binary_logloss: 0.103966
[11]	valid_0's binary_logloss: 0.0973031
[12]	valid_0's binary_logloss: 0.0913518
[13]	valid_0's binary_logloss: 0.0860305

[I 2023-03-12 23:41:32,775] Trial 0 finished with value: 0.033781726423032996 and parameters: {'max_depth': 8, 'num_leaves': 15}. Best is trial 0 with value: 0.033781726423032996.


[112]	valid_0's binary_logloss: 0.0337817
[113]	valid_0's binary_logloss: 0.0338006
[114]	valid_0's binary_logloss: 0.0337938
[115]	valid_0's binary_logloss: 0.0338226
[116]	valid_0's binary_logloss: 0.0337853
[117]	valid_0's binary_logloss: 0.0338186
[118]	valid_0's binary_logloss: 0.033824
[119]	valid_0's binary_logloss: 0.0338278
[120]	valid_0's binary_logloss: 0.0338431
[121]	valid_0's binary_logloss: 0.0338718
[122]	valid_0's binary_logloss: 0.0338504
[123]	valid_0's binary_logloss: 0.0338326
[124]	valid_0's binary_logloss: 0.0338076
[125]	valid_0's binary_logloss: 0.0338049
[126]	valid_0's binary_logloss: 0.0338086
[127]	valid_0's binary_logloss: 0.0338437
[128]	valid_0's binary_logloss: 0.0338354
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split 

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



...

[104]	valid_0's binary_logloss: 0.0338467
[105]	valid_0's binary_logloss: 0.0338601
[106]	valid_0's binary_logloss: 0.0338186
[107]	valid_0's binary_logloss: 0.0338441
[108]	valid_0's binary_logloss: 0.0338048
[109]	valid_0's binary_logloss: 0.0338073
[110]	valid_0's binary_logloss: 0.0337797
[111]	valid_0's binary_logloss: 0.0337557
[112]	valid_0's binary_logloss: 0.033754
[113]	valid_0's binary_logloss: 0.0337609
[114]	valid_0's binary_logloss: 0.0337779
[115]	valid_0's binary_logloss: 0.033793
[116]	valid_0's binary_logloss: 0.0337815
[117]	valid_0's binary_logloss: 0.0337942
[118]	valid_0's binary_logloss: 0.0338093
[119]	valid_0's binary_logloss: 0.0338018
[120]	valid_0's binary_logloss: 0.0338165
[121]	valid_0's binary_logloss: 0.0337976
[122]	valid_0's binary_logloss: 0.0337966
[123]	valid_0's binary_logloss: 0.0337538
[124]	valid_0's binary_logloss: 0.0337603
[125]	valid_0's binary_logloss: 0.0337398
[126]	valid_0's binary_logloss: 0.0337472
[127]	valid_0's binary_logloss: 0.03

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[1]	valid_0's binary_logloss: 0.246344
[2]	valid_0's binary_logloss: 0.212196
[3]	valid_0's binary_logloss: 0.188084
[4]	valid_0's binary_logloss: 0.169222
[5]	valid_0's binary_logloss: 0.153763
[6]	valid_0's binary_logloss: 0.140804
[7]	valid_0's binary_logloss: 0.12969
[8]	valid_0's binary_logloss: 0.119971
[9]	valid_0's binary_logloss: 0.11144
[10]	valid_0's binary_logloss: 0.103948
[11]	valid_0's binary_logloss: 0.0972823
[12]	valid_0's binary_logloss: 0.0913353
[13]	valid_0's binary_logloss: 0.0860128
[14]	valid_0's binary_logloss: 0.0812345
[15]	valid_0's binary_logloss: 0.0769102
[16]	valid_0's binary_logloss: 0.0730199
[17]	valid_0's binary_logloss: 0.0695035
[18]	valid_0's binary_logloss: 0.066314
[19]	valid_0's binary_logloss: 0.0634332
[20]	valid_0's binary_logloss: 0.0608132
[21]	valid_0's binary_logloss: 0.058439
[22]	valid_0's binary_logloss: 0.0562713
[23]	valid_0's binary_logloss: 0.0543146
[24]	valid_0's binary_logloss: 0.0525296
[25]	valid_0's binary_logloss: 0.050887

[I 2023-03-12 23:43:07,969] Trial 99 finished with value: 0.03400531240428388 and parameters: {'max_depth': 9, 'num_leaves': 16}. Best is trial 2 with value: 0.03361645695434466.


[92]	valid_0's binary_logloss: 0.0342776
[93]	valid_0's binary_logloss: 0.0343205
[94]	valid_0's binary_logloss: 0.0342696
[95]	valid_0's binary_logloss: 0.0341962
[96]	valid_0's binary_logloss: 0.0341771
[97]	valid_0's binary_logloss: 0.0341781
[98]	valid_0's binary_logloss: 0.0341518
[99]	valid_0's binary_logloss: 0.0341166
[100]	valid_0's binary_logloss: 0.0341123
[101]	valid_0's binary_logloss: 0.0341305
[102]	valid_0's binary_logloss: 0.0341206
[103]	valid_0's binary_logloss: 0.0340407
[104]	valid_0's binary_logloss: 0.0340115
[105]	valid_0's binary_logloss: 0.0340287
[106]	valid_0's binary_logloss: 0.0340116
[107]	valid_0's binary_logloss: 0.0340257
[108]	valid_0's binary_logloss: 0.0340159
[109]	valid_0's binary_logloss: 0.0340053
[110]	valid_0's binary_logloss: 0.0340235
[111]	valid_0's binary_logloss: 0.034044
[112]	valid_0's binary_logloss: 0.034022
[113]	valid_0's binary_logloss: 0.0340629
[114]	valid_0's binary_logloss: 0.0340725
[115]	valid_0's binary_logloss: 0.0340722
[1

In [419]:
study.best_trial.params

{'max_depth': 9, 'num_leaves': 12}

In [425]:
params = {
    'task': 'train', 
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':'binary_logloss', 
    'verbose':-10000000,
    'seed':42,
    'max_bin':128,
    'n_estimators':128,
    # 'n_estimators':trial.suggest_int("n_estimators", 800, 1200),
    'learning_rate':0.08,
    'feature_fraction':1.0,
    'bagging_fraction':1.0,
   # 'bagging_freq':trial.suggest_categorical("bagging_freq", [1]),
    'min_data_in_leaf':200,
    'min_gain_to_split':1.0,
}
    

In [426]:
params.update(study.best_trial.params)

In [427]:
tuned_model = lgb.LGBMClassifier(**params)
tuned_model.fit(X_train, y_train, 
        eval_set=[(X_val, y_val)],
        eval_metric=['binary_logloss'],
        early_stopping_rounds=100) 

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split is set=1.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[1]	valid_0's binary_logloss: 0.246399
[2]	valid_0's binary_logloss: 0.21223
[3]	valid_0's binary_logloss: 0.188108
[4]	valid_0's binary_logloss: 0.169264
[5]	valid_0's binary_logloss: 0.153825
[6]	valid_0's binary_logloss: 0.140813
[7]	valid_0's binary_logloss: 0.129724
[8]	valid_0's binary_logloss: 0.120032
[9]	valid_0's binary_logloss: 0.111515
[10]	valid_0's binary_logloss: 0.104015
[11]	valid_0's binary_logloss: 0.0973562
[12]	valid_0's binary_logloss: 0.0914111
[13]	valid_0's binary_logloss: 0.086076

LGBMClassifier(bagging_fraction=1.0, feature_fraction=1.0, learning_rate=0.08,
               max_bin=128, max_depth=9, metric='binary_logloss',
               min_data_in_leaf=200, min_gain_to_split=1.0, n_estimators=128,
               num_leaves=12, objective='binary', seed=42, task='train',
               verbose=-10000000)

In [440]:
X_test.shape

(11757, 36)

In [430]:
preds = tuned_model.predict_proba(X_test)
testloss = log_loss(y_test, preds)
testloss

0.030543711801184797

In [431]:
train_preds = tuned_model.predict_proba(X_train)
trainloss = log_loss(y_train, train_preds)
trainloss

0.02540742065869612

In [432]:
val_preds = tuned_model.predict_proba(X_val)
valloss = log_loss(y_val, val_preds)
valloss

0.03361645695434466

# Submission Pipeline

In [441]:
# read the real test data from test.csv
test_data = pd.read_csv("test.csv")

In [442]:
X_testtest = test_data.drop(['id'], axis=1)
ID_test = test_data.id

In [443]:
X_testtest = poly.fit_transform(X_testtest)

In [444]:
preds = tuned_model.predict_proba(X_testtest)

In [445]:
preds

array([[9.99515200e-01, 4.84800360e-04],
       [9.99230283e-01, 7.69716583e-04],
       [9.99826850e-01, 1.73149779e-04],
       ...,
       [9.99832612e-01, 1.67387506e-04],
       [9.35309601e-01, 6.46903993e-02],
       [1.61411435e-02, 9.83858856e-01]])

In [446]:
preds=  preds[:,1]

In [447]:
preds

array([4.84800360e-04, 7.69716583e-04, 1.73149779e-04, ...,
       1.67387506e-04, 6.46903993e-02, 9.83858856e-01])

In [448]:
preds.shape

(78377,)

In [449]:
# Save test predictions to file
submission = pd.DataFrame({'id': ID_test,
                       'Class': preds})
submission.to_csv('submission_optuna_lgbm.csv', index=False)